# 로지스틱 회귀 - 이항분류

## 로지스틱 회귀 분석 수행

### 1) 패키지 준비하기

In [ ]:
import sys
sys.path.append('../../')
import helper

from pandas import read_excel, DataFrame
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.model_Selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from tensorflow.keras.models import Sequential # 순서층을 구성하는 모델 객체 생성 기능
from tensorflow.keras.layers import Dense      # 모델 객체에 학습층을 쌓기 위한 클래스
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

### 2) 데이터셋 준비
- 방사선학 석사과정 대학원생을 모집하였다. 이 때 지원한 방사선사의 대학원 합격 여부에 따른 주요 요인이 무엇인지 분석하라
- 단, 독립 변수는 정규분포(normal distribution)를 따른다고 가정한다.

|변수|구분|설명|
|--|--|--|
|합격여부|범주형|1: 합격, 0: 불합격|
|필기점수|연속형|800점 만점|
|학부성적|연속형|4.0 만점|
|병원경력|범주형|1: 10년이상, 2: 5~9년, 3: 1~4년, 4: 1년 미만

In [ ]:
origin = read_excel('https://data.hossam.kr/E05/gradeuate.xlsx')
origin.head()

### 3) 데이터 전처리
- 결측치 확인

In [ ]:
origin.isna().sum()

- 명목형에 대한 데이터타입 설정

In [ ]:
origin['합격여부'] = origin['합격여부'].astype('category')
origin.info()

### 4) 탐색적 데이터 분석
- 기본통계 확인

In [ ]:
origin.describe()

- 상자그림 확인

In [ ]:
plt.figure(figsize=(10, 5))
sb.boxplot(data=origin)
plt.show()
plt.close()

> 데이터 단위가 크게 다르므로 표준화가 필요해 보임

### 5) 데이터셋 분할하기
- 랜덤시드 고정

In [ ]:
np.random.seed(777)

- 훈련 데이터(독립변수)와 레이블(종속변수) 구분하기

In [ ]:
x = origin[['필기점수', '학부성적','병원경력']]
y = origin[['합격여부']]
x.shape, y.shape

- 데이터 표준화

In [ ]:
x_scaler = StandardScaler()
x_scale = x_scaler.fit_transform(x)
print(x_scale.shape)
x_scale

- 표준화 전/후 상자그림 비교

In [ ]:
# 임시로 독립변수와 종속변수를 하나의 DataFrame으로 병합
x_scale_df = DataFrame(x_scale, columns=['필기점수', '학부성적', '병원경력'])

fig, ax = plt.subplots(1, 2, figsize=(15, 5), dpi=150)
sb.boxplot(data=origin, ax=ax[0])
sb.boxplot(data=x_scale_df, ax=ax[1])
ax[0].set_title('before')
ax[1].set_title('after')
plt.show()
plt.close()


- 훈련데이터와 검증데이터로 분할

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scale, 
                                                    y, 
                                                    test_size=0.3,
                                                    random_state = 777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### 6) 모델 개발
- 모델 정의
- [relu,] sigmoid rmsporp binary_crossentropy acc

In [ ]:
my_model = Sequential()
my_model.add(Dense(32, activation = 'relu', input_shape(3, )))
my_model.add(Dense(1, activation = 'sigmoid'))
my_model.compile(optimizer='rmsprop', 
                 loss='binary_crossentropy',
                 metrics=['acc'])
my_model.summary()

- 학습하기

In [ ]:
result = my_model.fit(x_train,
                      y_train, 
                      epochs=500,
                      validation_data = (x_test, y_test),
                      callbacks = [EarlyStopping(monitor='val_loss',
                                                 patience=5,
                                                 verbose=1),
                                   ReduceLROnPlateau(monitor='val_loss',
                                                     patience=3,
                                                     factor=0.5,
                                                     min_lr=0.0001,
                                                     verbose=1)])

### 7) 학습 결과 평가

In [ ]:
helper.tf_result_plot(result)

evaluate1 = my_model.evaluate(x_train, y_train)
print('최종 훈련 손실룰: %f, 최종 훈련 정확도: %f' % (evaluate1[0], evaluate1[1]))

evaluate2 = my_model.evaluate(x_test, y_test)
print('최종 검증 손실률: %f, 최종 검증 정확도: %f' % (evaluate2[0], evaluate2[1]))

### 8) 학습 결과 적용
- 훈련 데이터에 대한 예측 결과 산정

In [ ]:
train_pred = my_model.predict(x_train)
data_count, case_count = train_pred.shape
print('%d개의 훈련 데이터가 %d개의 경우의 수를 갖는다.' % (data_count, case_count))
print(train_pred)

- 검증 데이터에 대한 예측 결과 산정

In [ ]:
test_pred = my_model.predict(x_test)
data_count, case_count = train_pred.shape
print('%d개의 검증 데이터가 %d개의 경우의 수를 갖는다.' % (data_count, case_count))
print(test_pred)

- 결과 데이터셋 구성

In [1]:
kdf = DataFrame({
    '결과값': y_train['합격여부'].values,
    '예측값': np.round(train_pred.flatten()),
    '합격확률(%)': np.round(train_pred.flatten() * 100, 1)
})
kdf

NameError: name 'DataFrame' is not defined

- 혼돈행렬
    - 각 항목의 의미:
        |구분|설명|
        |--|--|
        |TN(True Negative, Negative Negative)|실제는 Negative인데, Negative로 예측함.|
        |FP(False Positive, Negative Positive)|실제는 Negative인데, Positive로 예측함. (Type1 Error)|
        |FN(False Negative, Positive Negative)|실제는 Positive인데, Negative로 예측함. (Type2 Error)|
        |TP(True Positive, Positive Positive)|실제는 Positive인데, Positive로 예측함.|

In [ ]:
cm = confusion_matrix(kdf['결과값'], kdf['예측값'])
tn, fp, fn, tp = cm.ravel()
print(tn, fp, fn, tp)
cmdf1 = DataFrame([[tn, tp], [fn, fp]], 
                  index=['True', 'False'], 
                  columns=['Negative', 'Positive'])
cmdf1

In [ ]:
cm2 = confusion_matrix(kdf['결과값'], kdf['예측값'])
cmdf2 = DataFrame(cm2, 
                  index=['실제(False)', '실제(True)'], 
                  columns=['예측(Negative)', '예측(Positive)'])

In [ ]:
plt.figure(figsize=(5, 3))
# 오차 행렬을 히트맵 그래프로 표현
# -> annot : 그래프의 각 칸에 수치값 출력
# -> fmt : 수치값 출력 format (여기서는 10진수)
# -> cmap : 색상맵 (<https://matplotlib.org/3.2.1/tutorials/colors/colormaps.html>)
sb.heatmap(cm,
           annot = True,
           fmt = 'd',
           cmap = 'Blues')
plt.xticks([0.5,1.5], ['Negative', 'Positive'])
plt.yticks([0.5,1.5], ['True', 'False'])
plt.xlabel('예측값')
plt.ylabel('결과값')
plt.show()

- 평가지표

In [ ]:
ras = roc_auc_score(kdf['결과값'], kdf['예측값'])
# 위양성율, 재현율, 임계값(사용안함)
fpr, tpr, thresholds = roc_curve(kdf['결과값'], kdf['예측값'])

plt.figure()
sb.lineplot(x=fpr, y=tpr)
sb.lineplot(x=[0,1],
            y=[0,1],
            color='red',
            linestyle=':',
            alpha=0.5)
plt.title('AUC={:.4f}'.format(ras))
plt.grid()
plt.show()
plt.close()

In [ ]:
# 정확도
acc = accuracy_score(kdf['결과값'], kdf['예측값'])

# 정밀도
pre = precision_score(kdf['결과값'], kdf['예측값'])

# 재현율
recall = recall_score(kdf['결과값'], kdf['예측값'])

# F1 score
f1 = f1_score(kdf['결과값'], kdf['예측값'])

# 위양성율
fallout = fp / (fp + tn)

# 특이성
spe = 1 - fallout

result_df = DataFrame({'정확도(Accuracy)': [acc],
                       '정밀도(Precision)': [pre],
                       '재현율(Recall, TPR)': [recall],
                       '위양성율(Fallout, FPR)': [fallout],
                       '특이성(Specificity, TNR)': [spe],
                       'RAS': [ras],
                       'f1_score': [f1]})

result_df

- 분류 보고서
    - precision : 정밀도 (분류기가 참으로 분류한 항목 중에서 실제 데이터가 참인 비율)
    - recall : 재현율 (실제 참 중에서 분류기가 참으로 분류한 비율)
    - f1-score : 2(Recall Precision) / (Recall + Precision)
        - Precision과 Recall의 조화평균
        - 주로 분류 클래스 간 데이터가 심각한 불균형을 이루는 경우에 사용
        - 정확도의 경우, 클래스가 데이터 분류가 균일하지 못하면 머신의 성능을 제대로 나타내줄 수 없기 때문에 F1을 사용한다.
    - support : 전체 데이터 수

In [2]:
report = classification_report(kdf['결과값'], kdf['예측값'])
print(report)

NameError: name 'classification_report' is not defined